# Numerics

Numbers are just numbers, right? That's not quite the case when it comes to numeric computation on computers.
In science, we deal with exact integers, inexact numbers, very large numbers (distance in Astronomy) and very small numbers (size in Particle Physics).
To cope with these disparate needs, computers have a number of ways of representing numbers to suite different needs, these representations have dedicated hardware resources for fast computations.

Julia has a broad range of primitive numeric types and a full complement of arithmetic/bitwise operators and mathematical functions.
Where possible, they map directly onto hardware resources thus allowing Julia to fully leverage modern computational capabilities.
Julia also provides software support for *Arbitrary Precision Arithmetic* but at the cost of slower performance.

## Number types

Julia has a type hierarchy where every type belongs.
Numeric types all fall under the **Number** type.
We can find the children of a type with the **subtypes()** function, and the parent of a type with the **supertype()** function.
Let's have a look at Number.

In [1]:
subtypes(Number)

2-element Array{Any,1}:
 Complex
 Real

Number consists of **Complex** and **Real**.

In [2]:
subtypes(Real)

4-element Array{Any,1}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

Real consists of **Integer**, **Rational**, **AbstractIrrational** and **AbstractFloat**.

The **supertype()** function tells us the parent of a type:

In [3]:
supertype(Integer)

Real

Here's a fast way to see the entire tree underneath Number:

In [4]:
using AbstractTrees
AbstractTrees.children(x::Type) = subtypes(x)
print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


As you can see, Julia has a rich assortment of numeric types to meet a wide variety of needs.
Whereas many dynamic languages won't let you control numeric types, Julia gives you the ability to control should you need to.
See [Integers and Floating-Point Numbers](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/) for full documentation.

Julia also defines two types **Int** and **UInt** which are aliases for the system's signed and unsigned native integer types respectively.

In [5]:
Int, UInt

(Int64, UInt64)

**BigInt** and **BigFloat** are arbitrary precision integer and floating point numbers.

## Limits

Each numeric type has a limit on what it can and cannot represent.
The range of valid values can be found via this code:

In [6]:
for T in [Int8, Int16, Int32, Int64, Int128, UInt8, UInt16, UInt32, UInt64, UInt128]
    println("$(lpad(T,7)): [$(typemin(T)), $(typemax(T))]")
end

   Int8: [-128, 127]
  Int16: [-32768, 32767]
  Int32: [-2147483648, 2147483647]
  Int64: [-9223372036854775808, 9223372036854775807]
 Int128: [-170141183460469231731687303715884105728, 170141183460469231731687303715884105727]
  UInt8: [0, 255]
 UInt16: [0, 65535]
 UInt32: [0, 4294967295]
 UInt64: [0, 18446744073709551615]
UInt128: [0, 340282366920938463463374607431768211455]


Note that integer arithmetic in Julia wraps around on overflow.

In [7]:
x = typemax(Int64)
(x, x+1)

(9223372036854775807, -9223372036854775808)

Floating point numbers have a *positive zero* and a *negative zero*, they are both just zero for computational purposes.

In [13]:
a = 0.0
b = -0.0
(a, b)

(0.0, -0.0)

In [14]:
a == b

true

## Numeric literals

Numeric literals are the string you enter to represent a numeric value.
Integer literals are just a sequence of digits without a decimal point.
You can insert _ (anywhere) to improve readability.

In [8]:
(1, 10, 1_000, 1_000_000)

(1, 10, 1000, 1000000)

Unsigned integer literals are usually entered using the **0x** prefix followed by hexadecimal digits.

Boolean literals uses the special names **true** and **false**.

Floating point literals have **decimal point** or **exponent** in the ususal form:

In [9]:
x = 123.56             # have decimal point
(x, typeof(x))

(123.56, Float64)

In [10]:
x = 123e3              # have exponent Exx - Float64
(x, typeof(x))

(123000.0, Float64)

In [11]:
x = 123f3              # have exponent Fxx - Float32
(x, typeof(x))

(123000.0f0, Float32)

The **typeof()** function is extremely useful as it tells us the type of a value.

## Numeric literal coefficients

Julia uses numeric literal coefficients to make an arithmetic expression look more similar to an algebraic equation.
You can place a numeric literal immediately before (no white space allowed) a variable or a parenthesized expression.
Julia will interpret it as an implied multiplication, this makes writing polynomials very clean:

In [12]:
x = 1
y = 3x^2 - 5(x-2) + 8

16